In [ ]:
from models.lenet import LeNet
from models.resnet import build_deep_resnet
from models.visiontransformer import VisionTransformer
from scripts.data_split import split_by_year
import optuna
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam, AdamW
from tensorflow.keras import metrics
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report

In [ ]:
data = np.load('data.npy')
df = pd.read_csv('metadata.csv')


In [ ]:
res = split_by_year(df, data)
X_train, df_train = res['train']
X_test, df_test = res['test']
X_val, df_val = res['val']
y_train = df_train['labels']
y_test = df_test['labels']
y_val = df_val['labels']

In [ ]:
early_stopping = EarlyStopping(
    monitor='val_loss',     
    patience=20,            
    restore_best_weights=True
)

In [ ]:
INPUT_SHAPE = (33, 33, 8)

In [ ]:
def objective(trial):
    initial_lr = trial.suggest_float("initial_lr", 1e-4, 1e-2, log=True)
    decay_steps = trial.suggest_int("decay_steps", 5000, 30000, step=5000)
    alpha = trial.suggest_float("alpha", 0.0, 0.2)
    batch_size = trial.suggest_categorical("batch_size", [16,32,64])
    weight_decay = trial.suggest_float("weight_decay", 1e-5, 1e-2, log=True)
    lr_schedule = keras.optimizers.schedules.CosineDecay(
        initial_learning_rate=initial_lr,
        decay_steps=decay_steps,
        alpha=alpha
    )

    optimizer = keras.optimizers.Adam(learning_rate=lr_schedule, weight_decay=weight_decay)

    vit = VisionTransformer()
    model = vit.build_model()

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
        loss='binary_crossentropy',
        metrics=['accuracy',
                ])
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=1000,
        class_weight=class_weights,
        batch_size=batch_size,
        callbacks=[early_stopping]
    )
    class_names = ['0', '1']
    loss, acc = model.evaluate(X_test, y_test)
    print(f"Test Accuracy: {acc:.4f}")

    preds = model.predict(X_test)
    preds = (preds > 0.5).astype(int)
    print(classification_report(y_test, preds, target_names=class_names))
    return acc